In [20]:
import random
from __future__ import division
import svmprediction
reload(svmprediction)
from features import extract_features
from features import encode_categorical_features
from sklearn import svm

(features, labels, _, _) = extract_features(2014, 2014)
(svm_features, encoder) = encode_categorical_features(features)

classifier = svm.SVC()
cm = svmprediction.plot_confusion_matrix(classifier,svm_features,labels)

29630
[[4954  248]
 [1863  343]]
Recall: 27.3287369811 %
Precision: 84.4514959202 %
Accuracy: 71.5037796976 %


In [96]:
#Example

#Train classifier
classifier.fit(svm_features, labels)  

#Predict result for play
X = defaultdict(float)
X['team'] = "GB"
X['opponent'] = "NYG"
X['time'] = 425
X['quarter'] = 4 
X['position'] = 10
X['half'] = 2
X['togo'] = 10
X['shotgun'] = 0
X['pass'] = 1
X['passlen'] = 'short'
X['side'] = 'left'
X['qbrun'] = 0
X = encoder.transform(X)

y_pred = clf.predict(X)
print prediction

KeyboardInterrupt: 